# Barcelona Total Housing

This is working code that cleans the property type data to give us the total number of housing properties by barri in Barcelona. It exports a dataframe of 2010-2019 total housing. 

The file is very clean and I'm quite proud of that. 


## Bring in the Dataframes

In [1]:
#import pandas for data stuff
import pandas as pd
#import geopandas for spatial data stuff
import geopandas as gpd

/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.1-CAPI-1.13.3) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
#start with the beginning dataset 
pdf = gpd.read_file('data/2010_Number of housing premises by type of owner.csv')

In [3]:
#trim the df to only include the columns needed; this will delete the gender titles as well.
pdf_trimmed = pdf[['Codi_Barri', 'Nom_Barri', 'Tipus_propietari', 'Nombre']]
pdf_trimmed.head()

,Codi_Barri,Nom_Barri,Tipus_propietari,Nombre
0,1,el Raval,Persona física. Nacionalitat espanyola,15024
1,2,el Barri Gòtic,Persona física. Nacionalitat espanyola,6587
2,3,la Barceloneta,Persona física. Nacionalitat espanyola,7074
3,4,"Sant Pere, Santa Caterina i la Ribera",Persona física. Nacionalitat espanyola,9564
4,5,el Fort Pienc,Persona física. Nacionalitat espanyola,11761


There are two datapoints that have NA, so I swapped them for 0s. Because the data is used for a total housing number, there were only two datapoints with NA, and the neighborhoods had smaller amounts of total housing otherwise, I felt comfortable doing so. 

In [4]:
#replace NA with 0
pdf_trimmed[pdf_trimmed.Nombre == 'NA'] = 0

<ipython-input-4-0fb278ea7e67>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_trimmed[pdf_trimmed.Nombre == 'NA'] = 0
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:2957: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:2935: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

## Group by Neighborhood

This data has total number of houses by the type of property. So, I'll group all of the properties for each neighborhood into the neighborhood.

In [5]:
#change datatype of Number to integer
pdf_trimmed['Nombre'] = pdf_trimmed['Nombre'].astype(int)

<ipython-input-5-a7295a670ec6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_trimmed['Nombre'] = pdf_trimmed['Nombre'].astype(int)


In [6]:
#group by code barri and name barri, summing the numbers of the sexes. 
pdfa= pdf_trimmed.groupby(['Codi_Barri', 'Nom_Barri']).sum()[['Nombre']]

In [7]:
#reset the index. flatten
pdfb = pdfa.reset_index()

In [8]:
pdfb.columns = ['c_barri', 'n_barri', '2010']

In [9]:
pdfb= pdfb.drop([0])

## Clean additional years. 

Now that I've figured out how to get the data how I want it for the initial year, I will bring in the additional dataframes, build a function to clean the rest of the data, and loop to combine all data into one dataframe. 

In [10]:
#bring in the data
pdf11 = gpd.read_file('data/2011_Number of housing premises by type of owner.csv')
pdf12 = gpd.read_file('data/2012_Number of housing premises by type of owner.csv')
pdf13 = gpd.read_file('data/2013_Number of housing premises by type of owner.csv')
pdf14 = gpd.read_file('data/2014_Number of housing premises by type of owner.csv')
pdf15 = gpd.read_file('data/2015_Number of housing premises by type of owner.csv')
pdf16 = gpd.read_file('data/2016_Number of housing premises by type of owner.csv')
pdf17 = gpd.read_file('data/2017_Number of housing premises by type of owner.csv')
pdf18 = gpd.read_file('data/2018_Number of housing premises by type of owner.csv')
pdf19 = gpd.read_file('data/2019_Number of housing premises by type of owner.csv')
pdf20 = gpd.read_file('data/2020_Number of housing premises by type of owner.csv')

In [11]:
#pass each df into a list 
years = [pdf11, pdf12, pdf13, pdf14, pdf15, pdf16, pdf17]

In [12]:
#create function for cleaning each dataframe
def clean(df):
    #grab year from 1st column 
    x=df.iloc[0][0]
    #trim the df
    pdf_trimmed = df[['Codi_Barri', 'Tipus_propietari', 'Nombre']]
    #change number to integer for the sum in groupby
    pdf_trimmed[pdf_trimmed.Nombre == 'NA'] = 0
    pdf_trimmed['Nombre'] = pdf_trimmed['Nombre'].astype(int) 
    pdfa= pdf_trimmed.groupby(['Codi_Barri']).sum()[['Nombre']]
    pdfb = pdfa.reset_index()
    #rename column titles
    pdfb.columns = ['c_barri' , str(x)]
    #show me the $$!!
    return pdfb

In [13]:
#create empty list for the new dfs
new_years = []
#LOOP whoop whoop
for df in years:
    new_years.append(clean(df))

<ipython-input-12-20d9fd426d8d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_trimmed[pdf_trimmed.Nombre == 'NA'] = 0
<ipython-input-12-20d9fd426d8d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_trimmed['Nombre'] = pdf_trimmed['Nombre'].astype(int)


## Clean the later years. 

Barcelona City Hall changed their data collection practices between 2017 and 2018. So, the later years' data has additional columns. I created an additional function to clean the later years, loop those, and then combine into the final dataframe. 

In [14]:
#create function for cleaning each dataframe AFTER BCH changed their data collection
def newclean(df):
    #grab year from 1st column 
    x=df.iloc[0][0]
    #create df of only the numbers rather than meter squared
    b=df.query("Concepte == 'Nombre'")
    #trim the df
    pdf_trimmed = b[['Codi_barri', 'Desc_tipus_propietari', 'Nombre']]
    #change number to integer for the sum in groupby
    pdf_trimmed[pdf_trimmed.Nombre == 'NA'] = 0
    pdf_trimmed['Nombre'] = pdf_trimmed['Nombre'].astype(int) 
    pdfa= pdf_trimmed.groupby(['Codi_barri']).sum()[['Nombre']]
    pdfb = pdfa.reset_index()
    #rename column titles
    pdfb.columns = ['c_barri' , str(x)]
    #show me the $$!!
    return pdfb

In [15]:
#create list of later years dataframes
lateryears = [pdf18, pdf19, pdf20]

In [16]:
#loop and scrub
for df in lateryears:
    new_years.append(newclean(df))

<ipython-input-14-e5cd6923d5c3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_trimmed[pdf_trimmed.Nombre == 'NA'] = 0
<ipython-input-14-e5cd6923d5c3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_trimmed['Nombre'] = pdf_trimmed['Nombre'].astype(int)


In [17]:
#grab df from new_years to merge to pdfb
final = pdfb.merge(new_years[0].merge(new_years[1].merge(new_years[2].merge(new_years[3].merge(new_years[4].merge(new_years[5].merge(new_years[6].merge(new_years[7].merge(new_years[8].merge(new_years[9], on='c_barri'), on='c_barri'), on='c_barri'),on='c_barri'),on='c_barri'),on='c_barri'),on='c_barri'),on='c_barri'),on='c_barri'), on='c_barri')
#take a peek at the final product
final.head()

,c_barri,n_barri,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,el Raval,23284,23279,23342,23446,23409,23432,23444,23454,23112,23112,23202
1,10,Sant Antoni,20137,20215,20200,20234,20255,20271,20281,20264,19484,19484,19488
2,11,el Poble Sec,19202,19145,19159,19196,19178,19189,19216,19228,18520,18520,18569
3,12,la Marina del Prat Vermell,893,893,881,906,744,798,798,798,776,776,776
4,13,la Marina de Port,12818,12950,13153,13157,13103,13154,13277,13277,13315,13334,13335


In [18]:
#make sure all of the data came in
final.shape

(73, 13)

In [19]:
#export as csv to data folder
final.to_csv(r'data/2010_2020_TotalHousing_Barri.csv', index = False)